In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm 
tqdm.pandas()

In [2]:
# Gets All Project into a Dataframe
! curl -o 'projects.html' --compressed 'https://abstracts.societyforscience.org/?__RequestVerificationToken=dTTGLynT6dfCfKf_cMfe6fWg-9eSye5BQbvXYfXKT2CvYzRVwFMKfvLsRk4djP1BEwA92M2XIgD_mIgaPTZrNIFrM7Fk_XMkwHO7xEWnqVM1&KeywordOrPhrase=&FinalistLastName=&Category=Any%20Category&FairCountry=Any%20Country&FairState=Any%20State&ISEFYear[0].ID=2021&ISEFYear[0].IsChecked=false&ISEFYear[1].ID=2020&ISEFYear[1].IsChecked=false&ISEFYear[2].ID=2019&ISEFYear[2].IsChecked=false&ISEFYear[3].ID=2018&ISEFYear[3].IsChecked=false&ISEFYear[4].ID=2017&ISEFYear[4].IsChecked=false&ISEFYear[5].ID=2016&ISEFYear[5].IsChecked=false&ISEFYear[6].ID=2015&ISEFYear[6].IsChecked=false&ISEFYear[7].ID=2014&ISEFYear[7].IsChecked=false&ISEFYear[8].ID=0&ISEFYear[8].IsChecked=true&ISEFYear[8].IsChecked=false&IsGetAllAbstracts=True' --globoff -X POST -H 'User-Agent: Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:98.0) Gecko/20100101 Firefox/98.0' -H 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8' -H 'Accept-Language: en-US,en;q=0.5' -H 'Accept-Encoding: gzip, deflate, br' -H 'Content-Type: application/x-www-form-urlencoded' -H 'Origin: https://abstracts.societyforscience.org' -H 'Connection: keep-alive' -H 'Referer: https://abstracts.societyforscience.org/?__RequestVerificationToken=dTTGLynT6dfCfKf_cMfe6fWg-9eSye5BQbvXYfXKT2CvYzRVwFMKfvLsRk4djP1BEwA92M2XIgD_mIgaPTZrNIFrM7Fk_XMkwHO7xEWnqVM1&KeywordOrPhrase=&FinalistLastName=&Category=Any%20Category&FairCountry=Any%20Country&FairState=Any%20State&ISEFYear[0].ID=2021&ISEFYear[0].IsChecked=false&ISEFYear[1].ID=2020&ISEFYear[1].IsChecked=false&ISEFYear[2].ID=2019&ISEFYear[2].IsChecked=false&ISEFYear[3].ID=2018&ISEFYear[3].IsChecked=false&ISEFYear[4].ID=2017&ISEFYear[4].IsChecked=false&ISEFYear[5].ID=2016&ISEFYear[5].IsChecked=false&ISEFYear[6].ID=2015&ISEFYear[6].IsChecked=false&ISEFYear[7].ID=2014&ISEFYear[7].IsChecked=false&ISEFYear[8].ID=0&ISEFYear[8].IsChecked=true&ISEFYear[8].IsChecked=false&IsGetAllAbstracts=True' -H 'Cookie: __RequestVerificationToken=D6iA1DR1LTp-M__B6DgqkgE0Fb5EATRFunk2DJaUIQLC_Gu-nXIymJHPbGoM1z6lizs8MZZvek8T-pd6pR0XtD8LBgn7D0OnQpmsMChGsZI1; ARRAffinity=f6cd621af62d07651208efd53149b8655115e06af8bad354e81f8c9c3ec73832; ARRAffinitySameSite=f6cd621af62d07651208efd53149b8655115e06af8bad354e81f8c9c3ec73832' -H 'Upgrade-Insecure-Requests: 1' -H 'Sec-Fetch-Dest: document' -H 'Sec-Fetch-Mode: navigate' -H 'Sec-Fetch-Site: same-origin' -H 'Sec-Fetch-User: ?1' --data-raw '__RequestVerificationToken=Zus64XOaGb1z-xWyPIje3K1erw2Z8egLcpxrPGqFPBjwDp9vKivLAcnyOlINwjsXFt5znOjeLS0PnklDgyYUsEbjToKkV79XMcTw9BYB7ws1&KeywordOrPhrase=&FinalistLastName=&Category=Any+Category&FairCountry=Any+Country&FairState=Any+State&ISEFYear%5B0%5D.ID=2021&ISEFYear%5B0%5D.IsChecked=false&ISEFYear%5B1%5D.ID=2020&ISEFYear%5B1%5D.IsChecked=false&ISEFYear%5B2%5D.ID=2019&ISEFYear%5B2%5D.IsChecked=false&ISEFYear%5B3%5D.ID=2018&ISEFYear%5B3%5D.IsChecked=false&ISEFYear%5B4%5D.ID=2017&ISEFYear%5B4%5D.IsChecked=false&ISEFYear%5B5%5D.ID=2016&ISEFYear%5B5%5D.IsChecked=false&ISEFYear%5B6%5D.ID=2015&ISEFYear%5B6%5D.IsChecked=false&ISEFYear%5B7%5D.ID=2014&ISEFYear%5B7%5D.IsChecked=false&ISEFYear%5B8%5D.ID=0&ISEFYear%5B8%5D.IsChecked=true&ISEFYear%5B8%5D.IsChecked=false&IsGetAllAbstracts=True'

with open('projects.html') as f:
    html = f.read()
    ids = re.findall("ProjectId=[0-9]+", html)
    ids = pd.Series(ids)

print(f"There are {len(ids)} projects")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1428k    0 1427k  100   791   160k     89  0:00:08  0:00:08 --:--:--  352k
There are 10828 projects


In [3]:
awards = pd.read_html('projects.html')[0]

awards.drop(['Year', 'Finalist Name(s)', 'Category'], axis=1, inplace=True)
awards.head()

,Project Title,Fair Country,Fair State,Fair Province,Awards Won
0,Dynamic Response of a Human Neck Replica to Ax...,United States of America,MN,NaN,NaN
1,The Effect of Nutrient Solution Concentration ...,United States of America,UT,NaN,NaN
2,Do Air Root Pruning Pots Accelerate Success in...,United States of America,LA,NaN,NaN
3,Insect-repelling Plants & New Organic Pesticide,United States of America,TX,NaN,NaN
4,How Do Different Factors Affect the Accuracy o...,United States of America,MN,NaN,NaN


In [4]:
def split_awards(award):
    if award:
        return str(award).split("  ") # Splits by double spaces
    return None

awards['Awards Won'] = awards['Awards Won'].apply(split_awards)

In [5]:
# Gets the abstract and additional project information from each page
s = requests.session()

def get_paper_data(id):
    try:
        root = "https://abstracts.societyforscience.org/Home/FullAbstract?"
        response = s.get(root + id)

        soup = BeautifulSoup(response.content, 'html.parser')

        # Gets the div whose class is row
        col = soup.find('div', attrs={'class': 'col-sm-12'})
        elements = col.findChildren()

        title = elements[0].text

        category = elements[4].text.split("Category:")[1]

        year = int(elements[7].text.split("Year:")[1])

        finalists = elements[10].text.replace("  ", "").replace("\n", "")
        schools = [x.split(")")[0] for x in finalists.split("(School: ")[1:]]
        schools = set(schools) # Removes duplicates

        abstract = elements[14].text.replace("Abstract:", "")

        return pd.Series({
            'title': title,
            'category': category,
            'year': year,
            'schools': schools,
            'abstract': abstract
        })
    except:
        return pd.Series({
            'title': None,
            'category': None,
            'year': None,
            'schools': None,
            'abstract': None
        })
   
database = ids.progress_apply(get_paper_data)

database.dropna(subset=['title'], inplace=True)
database.head()

100%|██████████| 10828/10828 [42:42<00:00,  4.23it/s]


,title,category,year,schools,abstract
0,Dynamic Response of a Human Neck Replica to Ax...,Energy: Physical,2014,{},Purpose: A human neck replica was made to simu...
1,The Effect of Nutrient Solution Concentration ...,Physics and Astronomy,2014,{},Studies comparing the mineral nutrition of hyd...
2,Do Air Root Pruning Pots Accelerate Success in...,Physics and Astronomy,2014,{},The purpose of my project was to determine whi...
3,Insect-repelling Plants & New Organic Pesticide,Environmental Engineering,2014,{},Organochlorine pesticides in agriculture are n...
4,How Do Different Factors Affect the Accuracy o...,Earth and Environmental Sciences,2014,{},The purpose of this experiment is to determine...


In [6]:
print(f"Length Before:{len(database.index):10d}")
database = database.join(awards.set_index('Project Title'), on='title')
print(f"Length After: {len(database.index):10d}")
database.head()

Length Before:     10828
Length After:      10858


,title,category,year,schools,abstract,Fair Country,Fair State,Fair Province,Awards Won
0,Dynamic Response of a Human Neck Replica to Ax...,Energy: Physical,2014,{},Purpose: A human neck replica was made to simu...,United States of America,MN,NaN,[nan]
1,The Effect of Nutrient Solution Concentration ...,Physics and Astronomy,2014,{},Studies comparing the mineral nutrition of hyd...,United States of America,UT,NaN,[nan]
2,Do Air Root Pruning Pots Accelerate Success in...,Physics and Astronomy,2014,{},The purpose of my project was to determine whi...,United States of America,LA,NaN,[nan]
3,Insect-repelling Plants & New Organic Pesticide,Environmental Engineering,2014,{},Organochlorine pesticides in agriculture are n...,United States of America,TX,NaN,[nan]
4,How Do Different Factors Affect the Accuracy o...,Earth and Environmental Sciences,2014,{},The purpose of this experiment is to determine...,United States of America,MN,NaN,[nan]


In [7]:
# Renames some columns
database.rename(columns={"Fair Country": "country", "Fair State": "State", "Fair Province": "Province", "Awards Won":"awards"}, inplace=True)

In [8]:
# Saves the file to CSV
database.to_csv("isef-database.csv", index=False)